# SparklyRGT: rGT functions for data manipulation and visualization

**About:** The purpose of this Notebook is to demonstrate how to summarize, manipulate, and plot rGT data using the sparklyRGT package. It serves as a reference point for data analysis, and should not be edited. 

**Contact:**
* Dexter Kim: dexterkim2000@gmail.com
* Brett Hathaway: bretthathaway@psych.ubc.ca

**Requirements**
* Familiarity with Jupyter Notebook (https://www.youtube.com/watch?v=jZ952vChhuI)
* Please read the `readme.md` file before starting
* The data must be an excel file from MEDPC2XL (trial by trial data) 
* The data, sparklyRGT.py file, and this notebook must all be in the same folder
    * See [Section 4](#Changing-your-working-directory) if you want to have your data stored in a different folder

## Table of contents
* Section 1: [Loading data into Python](#1\)-Load-data-into-Python)
* Section 2: Baseline & Acquisition Analysis
    * 2A: [Data cleaning and processing](#2A\)-Baseline-&-Acquisition-Analysis)
    * 2B: [Plotting](#2B\)-Baseline-&-Acquisition-Analysis:-Plotting)
* Section 3: Latin Square Analysis and Plotting
    * 3A: [Data cleaning and processing](#3A\)-Latin-Square-Analysis)
    * 3B: [Plotting](#3B\)-Latin-Square-Analysis:-Plotting)
* Section 4: [Choice rGT](#4\)-Choice-rGT)
* Section 5: [Miscellaneous](#5\)-Miscellaneous)



**Please run the following cell before starting!**

In [1]:
#MEDPC rat gambling task functions imports, will print "I am being executed!" if functional
import sparklyRGT as rgt

#main imports 
import os
import pandas as pd
import numpy as np

# plotting imports 
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# stats imports 
import scipy.stats as stats

#the following lines prevents pandas from giving unecessary errors and increases the number of rows you can view 
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows',100)

I am being executed!


***

# 1) Load data into Python

[back to top](#rGT-functions-for-data-manipulation-and-visualization)

* Write the names of the files that you want to load in as a list to `file_names`

`df = load_data(file_names, reset_sessions = False)` 
* creates a table similar to the excel sheet(s) you loaded in. (in the order established in `file_names`) 
* `df` stands for dataframe, and will store the data you load in as a table
* passing `reset_sessions = True`:
    * makes the session numbers start from 1 again (you may want to do this for baseline analysis or if you're loading in multiple projects)
    
`df = load_multiple_data(file_names, reset_sessions = False)` 
* loads in multiple cohorts (with the same subject numbers) and assigns them unique subject numbers (ex. subject 1 of cohort 1 --> subject 101) 



In [ ]:
file_names = ['BH07_raw_free_S29-30.xlsx'] 

df = rgt.load_data(file_names)

#load_data won't print the dataframe. Use the following function to view the top of your dataframe. 
#Note: it should look the exact same as your first excel file. 

df.head()


***
# 2A) Baseline & Acquisition Analysis

[back to top](#rGT-functions-for-data-manipulation-and-visualization)

Set your objects! These will be used in the rest of section 2A and 2B. Examples are left in for clarity.
* Assign the rat subject numbers for each group to `exp_group` and `control_group`
    * you can add additional groups - just ensure you also add them to the group_list and group_names
* Assign the names of your groups to `group_names`
    * as a dictionary
* Put your groups together into a list
    * **must** be in the same order as they are in group_names 
* For plotting:
    * Assign the title of the project to `title`
    * Assign the range of sessions you want to include in figures to `startsess` and `endsess`


In [ ]:
control_group = [3, 4, 5, 6, 9, 13, 14, 15, 17, 18, 23, 24, 27, 28, 30, 31] #In this example: Tg negative rats

exp_group = [1, 2, 7, 8, 11, 12, 16, 19, 20, 21, 22, 25, 26, 29, 32] #In this example: Tg positive rats

group_names = {0: 'Tg negative',
              1: 'Tg positive'} 

group_list = [control_group, exp_group]

#for plotting: 
title = 'Nigrostriatal activation during acquisition' 

startsess = 29 #first session you would like to include in figures
endsess = 30 #last session you would like to include in figures

## Data cleaning

### Check your session data
* `check_sessions` gives us a summary for each rat including session numbers, session dates and # of trials for each session.
* This allows us to see if there are any missing/incorrect session numbers, and if MED-PC exported all of the desired data into the Excel file you loaded in (`file_names`).  

In [ ]:
rgt.check_sessions(df)

### Dropping & editing session numbers
* `rgt.drop_sessions(df, [session numbers])`
    * pass the df you loaded in and the session number(s) you'd like to remove (as a list)
    * For example, to remove all data from session 28 and 29, I would write: `rgt.drop_sessions(df, [28, 29])`
    * Requirement: session number must exist in the session column of df
    
    
* `rgt.edit_sessions(df, orig_sess = [list], new_sess = [list], subs = 'all')`
    * write the original session numbers you want to remove in `orig_sess`, and the numbers you want to replace them with in `new_sess`, in the correct order 
    * For example, to change **all** 30s to 29s, and 31s to 30s, I would write: `rgt.edit_sessions(df2, orig_sess = [30, 31], new_sess = [29, 30], subs = "all")`
    * If you want to make edits, **for certain subjects**, I would assign the subject numbers to  `subs`. For example, I would write `subs = [17, 21]`
    

* **in both cases:** assign to a new object (i.e., `df2`), or if you are sure you want to make these edits, you can save over the original df (assign to `df`)


In [ ]:
df2 = rgt.drop_sessions(df, [28])

#example for edit_sessions: 
#df2 = rgt.edit_sessions(df2, orig_sess = [30, 31], new_sess = [29, 30], subs = "all")

#### Check that you dropped/edited the desired session(s)

In [ ]:
rgt.check_sessions(df2) 

## Data processing

### Calculate variables for each rat

`df_sum = rgt.get_summary_data(df, mode = 'Session')`

In df_sum, the rows represent subjects (rats 1 to n)

The columns are explained below:
* `##P#` represents the percent choice of each option. For example, `29P1` represents the percentage of times P1 was selected during the 29th session. 
* `risk##` represents the risk score for each session: (P1 + P2) - (P3 + P4) 
* `collect_lat##` represents the mean collect latency for each session
* `choice_lat##` represents the mean choice latency for each session 
* `omit##` represents the number of omissions for each session
* `trial##` represents the number of trials completed for each session
* `prem##` represents the percentage of premature responses for each session

In [ ]:
df_sum = rgt.get_summary_data(df2) #change to df instead of df2 if you didn't do any session editing
df_sum #prints the dataset 

### Missing data

Check out the `impute_missing_data()` function in the [Miscellaneous section](#Impute-missing-data-by-taking-the-mean)


### Get the risk status of the rats

`df_sum, risky, optimal = rgt.get_risk_status(df_sum, startsess, endsess)`


This code adds two columns to your df: `mean_risk` and `risk_status`


* `risk_status == 1` indicates a positive risk score (optimal) 
* `risk_status == 2` indicates a negative risk score (risky)
* `mean_risk` is the mean risk score averaged across the sessions between `startsess` and `endsess` for a given subject
    * You can change `startsess` and `endsess` by passing the session numbers instead. For example, `rgt.get_risk_status(df_sum, 28, 30)`
    * Requirement: `startsess` and `endsess` must be in the df
* `print(risky, optimal)` prints out 2 lists of rat subjects: the risky rats, and the optimal rats 

In [ ]:
df_sum, risky, optimal = rgt.get_risk_status(df_sum, startsess, endsess)

print(df_sum[['mean_risk','risk_status']]) 
print(risky, optimal) 

### Export your data to an Excel file 

`export_to_excel(df, groups = None, column_name = 'group', new_file_name = 'summary_data', asin = False)`

* `groups`(optional): pass the group_list defined at the beginning of this Notebook to add a column that specifies which experimental group each rat is in
    * numbers will be assigned according to their order in group_list (in this example: `Tg negative rats == 0`, `Tg positive rat == 1`)
* `column_name`: Assign a name to the column in the exported Excel file that will specify the control vs. experimental group. In this example: `tg_status`
* `new_file_name`: Assign the name of the **new** Excel file that you're exporting
* `asin`: False by default; passing `asin = True` will perform an arcsine transformation on percentage variables: P1-P4 and premature responding
    * can then be imported into SPSS for analysis

In [ ]:
rgt.export_to_excel(df_sum, groups = group_list, column_name = 'tg_status', new_file_name = 'BH07_free_S29-30.xlsx', asin = True)

## Calculate means and SEMs for your experimental groups

`mean_scores, SEM = rgt.get_means_sem(df_sum, groups = None, group_names = None)`

* `df_sum`: the summary dataframe you created above, with the variables calculated per rat
* `groups`: the list of groups created at the start of this section (`group_list`)
* `group_names`: the dictionary of group names created at the start of this section (`group_names`)

Output:
* `mean_scores`: each value is the mean for that variable (ex. `29P1`) for each experimental group (in this example: `tg negative` and `tg positive`) 
* `SEM`: each value is the standard error of the mean for that variable for each experimental group
* If you want to view certain columns, specify them as a list in square brackets following `mean_scores` or `SEM`
    * For example, `mean_scores[['risk29', 'risk30']]` will display a table with only those 2 columns


These can also be exported to an excel file, using `mean_scores.to_excel('file name.xlsx')` or `SEM.to_excel('file name.xlsx')`
* check out [this link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html) for more details


In [ ]:
mean_scores, SEM = rgt.get_means_sem(df_sum, groups = group_list, group_names = group_names)
mean_scores #view all mean scores (truncated)
# mean_scores[['risk29', 'risk30']] #view certain columns


### Calculate means and SEMS for your experimental groups, split by risk status and/or sex

* Can use a technique called list comprehension to create lists of experimental risky rats, experimental optimal rats, etc.
    * example given below
    
    
* Alternatively, you can just manually type in the lists: 
    * e.g., `control_risky` = [1,2,3]
    
    
* You can adapt this code to split by sex, or further split risk groups by sex:
    * First, create a list of female and male subjects: `female = [1,2,3]`; `male = [4,5,6]`
    * `control_female = [subject for subject in control_group if subject in female]`
    * `control_risky_female = [subject for subject in control_risky if subject in female]`


* You can create as many groups as you like, just make sure you add them to `group_list_risk` and `group_names_risk`!

In [ ]:

control_risky = [subject for subject in control_group if subject in risky]
exp_risky = [subject for subject in exp_group if subject in risky]

control_optimal = [subject for subject in control_group if subject in optimal]
exp_optimal = [subject for subject in exp_group if subject in optimal]


group_list_risk = [control_risky,exp_risky, control_optimal, exp_optimal]

#make sure the group names are in the same order as the group list!
group_names_risk = {0:'Control risky', 
                    1: 'Experimental risky',
                    2: 'Control optimal',
                    3: 'Experimental optimal'}

mean_scores_risk, SEM_risk = rgt.get_means_sem(df_sum, group_list_risk, group_names_risk)

mean_scores_risk

    

# 2B) Baseline & Acquisition Analysis: Plotting

[back to top](#rGT-functions-for-data-manipulation-and-visualization)

**For all plotting functions**: If you are missing data for a particular rat, error bars will not appear on the figures

## Bar plot of P1-P4 % choice

`rgt.choice_bar_plot(startsess, endsess, mean_scores, SEM)`

* This function plots the mean P1-P4 choices for the experimental groups 
    * You can change startsess and endsess to calculate the mean over whichever sessions you like

In [ ]:
rgt.choice_bar_plot(startsess, endsess, mean_scores, SEM)

#to save this figure (or any other figure):
plt.savefig('BH07 Choice S29-30',facecolor = 'white')

## Line plot of other variables

`rgt.rgt_plot('variable', startsess, endsess, title, mean_scores, SEM, group_names = group_names, y_label = 'variable name', highlight = None)`
* `variable`: specifies the variable you want to plot. 
    * For example, if I want to plot `choice_lat`, I would replace `variable` with `'choice_lat'` (just the variable name, don't include session numbers)
* `startsess` and `endsess`: specifies the range of session numbers you'd like to plot 
    * For example, if I want to plot `choice_lat` over sessions 29 to 31, I would replace `startsess` and `endsess` with `29` and `31` respectively
    * Requirement: `startsess` and `endsess` must be in the df
* `group_names`: pass a dictionary with the `mean_scores` index for each group as the keys and the names of groups as the values (defined at beginning of this Notebook)
* `y_label`: specifies the Y axis label
* `highlight`: specify a session number - a line will be added at that session and the figure will be shaded gray to the right of the line
    * useful if you take your rats off of a drug or otherwise change your experimental manipulation, and want to represent that on the figure

In [ ]:
rgt.rgt_plot('risk', startsess, endsess, title, mean_scores, SEM, group_names = group_names, y_label = 'Risk score') 

## Bar plot of other variables

`rgt.rgt_bar_plot('variable', startsess, endsess, title, mean_scores, SEM, group_names = group_names, y_label = 'Variable name')`

* takes the mean of specified `variable` over session range specified by `startsess` and `endsess`
* plots as bar plot for each group specified in `group_names` dictionary
* `y_label`: Y axis label

In [ ]:
rgt.rgt_bar_plot('risk', startsess, endsess, title, mean_scores, SEM, group_names, y_label = 'Risk score')

## Plotting by risk status

### Bar plot of P1-P4 % Choice

* Use the functions above but pass `group_names_risk`, `mean_scores_risk` and `SEM_risk` instead

In [ ]:
rgt.choice_bar_plot(startsess, endsess, mean_scores_risk, SEM_risk)

In [ ]:
rgt.rgt_plot('risk', startsess, endsess, title, mean_scores_risk, SEM_risk, group_names = group_names_risk, y_label = 'Risk score') 

In [ ]:
rgt.rgt_bar_plot('prem', startsess, endsess, title, mean_scores_risk, SEM_risk, group_names = group_names_risk,y_label = 'Premature responding')

***
# 3A) Latin Square Analysis

[back to top](#rGT-functions-for-data-manipulation-and-visualization)

**This section assumes you have assigned dosing information to the 'Group' variable in MEDPC**
* in this example: vehicle = 1 in Group column, low dose = 2, mid dose = 3, high dose = 4

Set your objects! These will be used in the rest of section 3A and 3B. Examples are left in for clarity.
* Assign the names of the files that you want to analyze to `file_names`
* Assign a title, for plotting
* Assign the startdose and enddose variables - 1 and 4 in this example 

**Note:** This section assumes all of your rats received the same manipulation; if you have multiple experimental groups (e.g., different brain regions, Tg+/Tg-) in your dataset, see [Miscellaneous section](#Example:-between-subjects-analysis-for-Latin-Square-dataset) for an example of how to do so

In [15]:
file_names = ['BH06_raw_round1-infusions.xlsx', 'BH06_raw_round1-makeup.xlsx'] 

df = rgt.load_data(file_names)

#for plotting: 
title = '5-HT2c Antagonist' 

#lowest group number
startdose =  1
#highest group number
enddose = 4

## Data cleaning

### Check the Group numbers for each rat

`rgt.check_groups(df)`
* This function will show you the Group numbers and trials completed for each rat

In [ ]:
rgt.check_groups(df)

### Drop subjects & edit Group numbers

`df2 = rgt.edit_groups(df, orig_group = [list], new_group = [list], subs = 'all')`

* `orig_group`: enter the group number(s) that you want to change in square brackets 
* `new_group`: enter the new group number(s) you want to change to 
    * pay attention to order: first group number in `orig_group` will be changed to first group number in `new_group`, and so on for additional numbers    
* `subs`: by default this equals 'all' (changes will be made for all subjects)
    * pass a list of subject numbers if you only want to change the group numbers for certain subjects

`df2 = rgt.drop_subjects(df, subs = [list])`

* `subs`: enter a list of subjects that you want to remove the data for

**in both cases:** assign to a new object (i.e., `df2`), or if you are sure you want to make these edits, you can save over the original df (assign to `df`)


In [16]:
df2 = rgt.edit_groups(df, orig_group = [0], new_group = [3], subs = [5])

df2 = rgt.drop_subjects(df, subs = [7])


### Check that you edited the Group number/dropped subjects as desired

In [ ]:
rgt.check_groups(df2)

## Data processing

### Calculate variables for each rat at each dose

`df_sum = rgt.get_summary_data(df, mode = 'Group')`
* **for Latin Square data, make sure you pass `mode = 'Group'` to this function (will calculate by Session by default)**

The rows represent subjects (rats 1 to n)

The columns are explained below:
* `##P#` represents the percent choice of each option. For example `1P1` represents the percent choice of P1 at dose 1
* `risk##` represents the risk score for each dose: (P1 + P2) - (P3 + P4) 
* `collect_lat##` represents the mean collect latency for each dose
* `choice_lat##` represents the mean choice latency for each dose
* `omit##` represents the number of omissions for each dose
* `trial##` represents the number of trials completed for each dose
* `prem##` represents the number of premature responses for each dose

In [20]:
df_sum = rgt.get_summary_data(df2, mode = 'Group')
df_sum

I am being executed!


,1P1,1P2,1P3,1P4,2P1,2P2,2P3,2P4,3P1,3P2,...,omit3,omit4,trial1,trial2,trial3,trial4,prem1,prem2,prem3,prem4
1,0,34,64,2,0,57.1429,35.0649,7.79221,5,40,...,0,2,50.1,79.1,40.0,111.0,54.954955,32.478632,70.588235,14.615385
2,3.8961,59.7403,7.79221,28.5714,12.3288,61.6438,6.84932,19.1781,8.57143,45.7143,...,0,1,77.0,74.0,70.0,94.0,23.000000,28.846154,31.372549,17.543860
3,0,18,78,4,0,13.8889,80.5556,5.55556,2.22222,20,...,0,0,50.1,38.1,45.0,40.0,57.264957,70.400000,64.000000,56.989247
4,1.23457,67.9012,0,30.8642,1.21951,65.8537,0,32.9268,0,77.6596,...,1,2,86.0,82.0,95.0,79.0,24.561404,21.904762,16.666667,15.957447
5,5.12821,35.8974,55.1282,3.84615,3.84615,61.5385,28.2051,6.41026,17.4419,40.6977,...,1,0,79.1,79.0,87.0,128.0,33.898305,41.044776,21.621622,19.496855
6,1.62602,85.3659,0,13.0081,2.06186,72.1649,0,25.7732,1.90476,69.5238,...,0,0,125.1,98.0,105.0,121.0,13.888889,15.517241,10.256410,1.626016
8,4,44,0,52,6.89655,31.0345,5.17241,56.8966,0,42.8571,...,0,0,75.0,58.0,57.1,66.0,18.478261,42.574257,47.169811,41.071429
9,0,1.5625,98.4375,0,0,14.9254,82.0896,2.98507,1.85185,11.1111,...,1,2,65.1,69.0,55.0,48.0,26.436782,24.175824,46.078431,54.716981
11,3.63636,1.81818,94.5455,0,1.72414,3.44828,93.1034,1.72414,1.63934,4.91803,...,0,0,56.1,58.0,61.0,55.0,35.294118,31.764706,11.594203,29.487179
12,1.75439,66.6667,12.2807,19.2982,NaN,NaN,NaN,NaN,6.25,68.75,...,0,0,63.0,NaN,80.0,95.1,18.181818,NaN,35.483871,20.833333


### Impute missing data
`rgt.impute_missing_data(df, session = None, subject = None, choice = 'all', vars = 'all')`

* This function is described in the [Miscellaneous section](#Impute-missing-data-by-taking-the-mean)


In [ ]:
df_sum = rgt.impute_missing_data(df_sum, session = 2, subject = 12, choice = 'all', vars = 'all')

### Get risk status based on vehicle dose data

`df_sum,risky,optimal = rgt.get_risk_status_vehicle(df_sum)`

This function adds a column to your df: `risk_status`, calculated from the Vehicle dose data

* Note: 
    * `risk_status == 1` indicates a positive risk score (optimal) 
    * `risk_status == 2` indicates a negative risk score (risky)
    * `print(risky, optimal)` prints out 2 lists of rat subjects: the risky rats, and the optimal rats 


In [ ]:
df_sum,risky,optimal = rgt.get_risk_status_vehicle(df_sum)
print(risky, optimal)

### Export your data to an Excel file!

`export_to_excel(df, groups = None, column_name = 'group', new_file_name = 'summary_data', asin = False)`

* In this example: only one experimental condition, so optional groups and column_name arguments are not included
* `new_file_name`: Assign the name of the **new** Excel file that you're exporting
* `asin`: False by default; passing `asin = True` will perform an arcsine transformation on percentage variables: P1-P4 and premature responding
    * can then be imported into SPSS for analysis 

In [ ]:
rgt.export_to_excel(df_sum, new_file_name = 'BH06_all-data.xlsx', asin = True)

## Calculate means and SEMs for each dose

`mean_scores, SEM = rgt.get_means_sem(df_sum, groups = None, group_names = None)`

* `df_sum`: the summary dataframe you created above, with the variables calculated per rat
* `groups`: None by default, so doesn't need to be included 
* `group_names`: None by default, so doesn't need to be included
    * if not included, group name will be automatically assigned as 'All rats'

Output:
* `mean_scores`: each value is the mean for that variable (ex. `29P1`) for all rats 
* `SEM`: each value is the standard error of the mean for that variable for all rats
* If you want to view certain columns, specify them as a list in square brackets following `mean_scores` or `SEM`
    * For example, `mean_scores[['risk1', 'risk2']]` will display a table with only those 2 columns


These can also be exported to an excel file, using `mean_scores.to_excel('file name.xlsx')` or `SEM.to_excel('file name.xlsx')`
* check out [this link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html) for more details

In [ ]:
mean_scores, SEM = rgt.get_means_sem(df_sum)
mean_scores
# mean_scores[['omit3', 'omit4']]

### Calculating means and SEMS separately for risky and optimal rats

* you can pass risky and optimal lists, along with the group names, to the `get_means_sem` function to calculate mean/SEM values separately

In [ ]:

groups_risk = [risky,optimal]
group_names_risk = {0: 'Risky',
                    1: 'Optimal'}

means_risk, SEM_risk = rgt.get_means_sem(df_sum, groups = groups_risk, group_names = group_names_risk)

means_risk


# 3B) Latin Square Analysis: Plotting

[back to top](#rGT-functions-for-data-manipulation-and-visualization)

**For all plotting functions**: If you are missing data for a particular rat, error bars will not appear on the figures

## Bar plot for P1-P4 % Choice

`rgt.ls_bar_plot('group name',mean_scores,SEM)`

* `'group name'`: replace with the group name that you would like to plot
    * assigned as 'All rats' if group names are not passed to the function `rgt.get_means_SEM`

In [ ]:
rgt.ls_bar_plot('All rats',mean_scores,SEM)

## Line plot of other variables

`rgt.rgt_plot('variable', startsess, endsess, title, mean_scores, SEM, y_label = 'variable', x_label = 'Session')`
* `variable`: specifies the variable you want to plot. 
    * For example, if I want to plot `choice_lat`, I would replace `variable` with `'choice_lat'` (just the variable name, don't include session numbers)
* `startsess` and `endsess`: specifies the range of session numbers you'd like to plot 
    * **For Latin Square analysis**: We will pass the start and end Dose numbers (assigned to Group in MEDPC) - 1 and 4 in this example (startdose and enddose defined at beginning of this section)
* `y_label`: specifies the Y axis label
* `x_label`: specifies the X axis label; 'Session' by default, but here we want it to be 'Dose'


In [ ]:
rgt.rgt_plot('risk',startdose,enddose,title,mean_scores,SEM,y_label = 'Risk score', x_label = 'Dose')

## Plotting by risk status

for `rgt.ls_bar_plot('group', means, SEM)`:
* pass 'Risky' or 'Optimal' as the first argument, instead of 'All rats'
* use `means_risk` and `SEM_risk` instead

In [ ]:
rgt.ls_bar_plot('Risky',means_risk,SEM_risk)
rgt.ls_bar_plot('Optimal',means_risk,SEM_risk)

### Line plots for other variables by risk status

`rgt.rgt_plot('variable',startdose,enddose,title,means,SEM,group_names = None, y_label = 'variable', x_label = 'Dose')`

* pass an additional argument to `rgt.rgt_plot`: group_names = group_names_risk
* use means_risk and SEM_risk instead 
* assign `x_label` as 'Dose' (`x_label = 'Session'` by default)


In [ ]:

rgt.rgt_plot('risk',startdose,enddose,title,means_risk,SEM_risk,group_names = group_names_risk, y_label = 'Risk score', x_label = 'Dose')


***
# 4) Choice rGT

[back to top](#SparklyRGT:-rGT-functions-for-data-manipulation-and-visualization)

In [2]:
file_names = ['KH10B-FL-raw.xlsx'] 

df = rgt.load_data(file_names)

#load_data won't print the dataframe. Use the following function to view the top of your dataframe. 
#Note: it should look the exact same as your first excel file. 

df.head()

,MSN,StartDate,StartTime,Subject,Group,Box,Experiment,Comment,Session,Trial,...,Premature_Resp,Premature_Hole,Rew_Persev_H1,Rew_Persev_H2,Rew_Persev_H4,Rew_Persev_H5,Lever_Latency,Uncued_Chosen,Cued_Chosen,Choice_Omit
0,ChoicerGT_A-CR_gold,02/19/21,11:49:04,1,0.0,1,0.0,NaN,1,1.0,...,0,0,0,0,0,0,0.00,0,0,1
1,ChoicerGT_A-CR_gold,02/19/21,11:49:04,1,0.0,1,0.0,NaN,1,2.0,...,0,0,0,0,0,0,2.57,0,1,0
2,ChoicerGT_A-CR_gold,02/19/21,11:49:04,1,0.0,1,0.0,NaN,1,3.0,...,0,0,0,0,0,0,0.00,0,0,1
3,ChoicerGT_A-CR_gold,02/19/21,11:49:04,1,0.0,1,0.0,NaN,1,4.1,...,1,5,0,0,0,0,2.44,0,1,0
4,ChoicerGT_A-CR_gold,02/19/21,11:49:04,1,0.0,1,0.0,NaN,1,4.0,...,0,0,0,0,0,0,2.33,0,1,0


In [3]:
males = list(range(1,33))

females = []

group_names = {0: 'males',
              1: 'females'} 

group_list = [males,females]

#for plotting: 
title = 'Choice rGT' 

startsess = 1 #first session you would like to include in figures
endsess = 15 #last session you would like to include in figures


In [14]:
df = rgt.get_choices(df)
df_sum = rgt.get_sum_choice_all(df, task = 'choiceRGT')
df_sum
df_sum = rgt.get_latencies(df, df_sum, task = 'choiceRGT')
df_sum
df_sum = rgt.get_preference_score(df, df_sum)
df_sum

,1_cued_P1,1_cued_P2,1_cued_P3,1_cued_P4,1_uncued_P1,1_uncued_P2,1_uncued_P3,1_uncued_P4,2_cued_P1,2_cued_P2,...,pref6,pref7,pref8,pref9,pref10,pref11,pref12,pref13,pref14,pref15
1,22.7273,22.7273,43.1818,11.3636,43.4783,4.34783,26.087,26.087,41.1765,25.4902,...,0.513089,0.597403,0.517413,0.528736,0.489796,0.515152,0.532051,0.510067,0.477987,0.535948
2,48,40,4,8,61.5385,7.69231,7.69231,23.0769,55.2632,34.2105,...,0.561404,0.418750,0.597315,0.514286,0.515152,0.507937,0.515152,0.544910,0.497207,0.493902
3,28,36,8,28,72.7273,0,21.2121,6.06061,59.5745,10.6383,...,0.470968,0.478528,0.446237,0.527778,0.502674,0.552486,0.486339,0.567251,0.470588,0.474576
4,38.806,28.3582,25.3731,7.46269,50,13.3333,13.3333,23.3333,20,20,...,0.476190,0.535519,0.476744,0.502732,0.511962,0.491979,0.457895,0.477778,0.451282,0.478022
5,23.8095,33.3333,14.2857,28.5714,60.7143,25,3.57143,10.7143,40,32,...,0.540984,0.550000,0.788462,0.619048,0.565789,0.612903,0.465517,0.531915,0.387500,0.488372
6,80.7018,3.50877,7.01754,8.77193,20.5882,29.4118,11.7647,38.2353,10.7143,46.4286,...,0.533679,0.491803,0.473958,0.487923,0.528205,0.500000,0.548387,0.731884,0.480952,0.484375
7,37.5,17.5,15,30,38.4615,30.7692,7.69231,23.0769,72.9167,20.8333,...,0.551724,0.557823,0.533784,0.530120,0.548193,0.556338,0.564626,1.000000,0.550336,0.617391
8,36.8421,21.0526,18.4211,23.6842,50,36.3636,2.27273,11.3636,23.3333,38.3333,...,0.514706,0.464286,0.539216,0.413408,0.540404,0.537688,0.579545,0.558140,0.514423,0.452261
9,28.125,21.875,46.875,3.125,25.7143,14.2857,51.4286,8.57143,8.77193,50.8772,...,0.523810,0.500000,0.500000,0.582353,0.517483,0.497653,0.548837,0.541126,0.508850,0.547619
10,18.3099,30.9859,46.4789,4.22535,45.7143,30,18.5714,5.71429,13.2353,67.6471,...,0.556075,0.516432,0.513834,0.493333,0.489362,0.475285,0.528736,0.490842,0.517857,0.580786


In [ ]:
# df_sum.to_excel('test.xlsx')

***
# 5) Miscellaneous

[back to top](#rGT-functions-for-data-manipulation-and-visualization)

## Changing your working directory

* Check your current working directory by running `os.getcwd()` 
    * by default, this will be wherever the jupyter notebook file is saved
* After loading in rgt_functions.py, you may want to move to the folder where your data is saved
* Change your working directory using `os.chdir('C:\\...)`
    * can be absolute or relative file path
* For example, my current working directory is `'C:\\Users\\dexte\\hathaway_1'`, I can change this by running `os.chdir('C:\\Users\\dexte\\hathaway_1\\data')` 
    * slashes will be different if you are not using Windows
* Data files can now be loaded in from and saved to the folder you moved to

In [ ]:
#checks current working directory
os.getcwd()

#changes working directory to whatever is included in brackets
os.chdir('C:\\Users\\dexte\\hathaway_1\\data') 

## Impute missing data by taking the mean

`rgt.impute_missing_data(df, session = None, subject = None, choice = 'all', vars = 'all')`

* this function takes the mean of the session before and after the missing data, for the specified session and subject
    * You can also specify a Group number in the session variable for Latin Square analysis
* For example, if you have missing data for subject 12, dose 2, this function will calculate the mean of dose 1 and 3
    * Code: `df = rgt.impute_missing_data(df, session = 2, subject = 12, choice = 'all', vars = 'all')`
* You can specify which P1-P4 options you'd like to impute by passing a list to `choice`
    * `choice = [1,3]`
* You can specify which variables by passing the variable names as a list to `vars`
    * `vars = ['prem', 'choice_lat']`

## Making changes to the rgt_functions.py file

* If you make changes in the .py file, run the following cell
* Every time you change & save the .py file, the changes will automatically be loaded into the Notebook!

In [6]:
%load_ext autoreload
%autoreload 2

## Example: between-subjects analysis for Latin Square dataset

* This example combines information from Section 2 and Section 3
* See Section 2 for guidance on how to export to excel, further split by risky/optimal, etc while taking experimental group information into account

### Define objects

This is similar to defining experimental groups in 2A:
* `group_names`: create a dictionary for your group names (here, I'm specifying different brain regions, but you could also specify any number of different experimental groups, e.g., Tg positive and Tg negative, as done in 2A) 
* `group_list`: create lists of subject numbers for your different groups, then put them together into group_list
* load in data using the `load_data(file_names)` function, as described in Section 1

In [ ]:
group_names = {0:'lOFC',
               1:'PrL'} 

lOFC = [1,2,3,4,5,6,8,9,11,12,13,14,15,16,24,25,26,32]

PrL = [18,19,20,21,22,23,27,28,29,30,31]

group_list = [lOFC, PrL]

file_names = ['BH06_raw_round1-infusions.xlsx',
              'BH06_raw_round1-makeup.xlsx',
              'BH06_raw_round2-infusions.xlsx']

#for plotting:
startdose = 1
enddose = 4
title = '5-HT2c Antagonist'


In [ ]:
df = rgt.load_data(file_names)
df = rgt.edit_groups(df, orig_group = [0], new_group = [3], subs = [5])

### Calculate descriptive statistics

* This is very similar to Section 3A, except that you are passing your group_list and group_names to the `rgt.get_means_sem` function

In [ ]:
df_sum = rgt.get_summary_data(df, mode = 'Group')

df_sum = rgt.impute_missing_data(df_sum, session = 2, subject = 12, choice = 'all', vars = 'all')

mean_scores, SEM = rgt.get_means_sem(df_sum, groups = group_list, group_names = group_names)



### Plotting

* Again, this is similar to 3B, except that you will pass the group names for the Choice bar plot instead of 'All rats'
* And pass `group_names = group_names` to the `rgt.rgt_plot()` function

In [ ]:
rgt.ls_bar_plot('lOFC',mean_scores,SEM)
rgt.ls_bar_plot('PrL',mean_scores,SEM)

In [ ]:
rgt.rgt_plot('prem',startdose,enddose,title,mean_scores,SEM, group_names = group_names, y_label = 'Premature response',x_label = 'Dose')